In [1]:
import numpy as np
from tools import *

In [2]:
def get_topk_acc(labels_q, labels_db, dists, topk):
        return sum(get_topk_hits(labels_q, labels_db, dists, topk)) / len(labels_q)


def get_topk_hits(labels_q, labels_db, dists, topk):
    indices = np.argsort(dists, axis=1)
    top_labels = np.array(labels_db)[indices[:, :topk]]
    hits = (top_labels.T == labels_q).T
    return np.sum(hits[:, :topk+1], axis=1) > 0

def get_top_ks(q_pids, distmat, ks=[1,3,5,10]):
    return [(k, get_topk_acc(q_pids, q_pids, distmat, k)) for k in ks]


In [3]:
config_path = 'configs/config_whaleshark.yaml'

from preprocess import load_to_df
config = get_config(config_path)
annotation_file = config['data']['annotation_file']
images_dir = config['data']['images_dir']

data_params = config['data']
species = config['species']

embeddings, uuids = load_pickle(data_params['embedding_file'])

df = load_to_df(annotation_file, format='old')

df['image_path'] = images_dir + '/' + df['file_name']


uuid_to_imagepath = df.set_index('uuid_x')['image_path'].to_dict()
uuid_to_name = df.set_index('uuid_x')['name_viewpoint'].to_dict()  
uuid_to_bbox = df.set_index('uuid_x')['bbox'].to_dict()  





Merging on image uuid
** Loaded /ekaterina/work/data/whaleshark_full/whaleshark.coco/annotations/whaleshark_singletons_flipped.json **
      Found 997 annotations


In [4]:
results = load_pickle("lightglue_scores_superpoint.pickle")

In [5]:
results

{'scores': array([[0.        , 0.21489265, 0.16561129, ..., 0.11232495, 0.20345116,
         0.16409038],
        [0.21489265, 0.        , 0.1521418 , ..., 0.        , 0.14139676,
         0.15541689],
        [0.16561129, 0.1521418 , 0.        , ..., 0.21432184, 0.13428774,
         0.26498961],
        ...,
        [0.11232495, 0.        , 0.21432184, ..., 0.        , 0.        ,
         0.22892763],
        [0.20345116, 0.14139676, 0.13428774, ..., 0.        , 0.        ,
         0.16970645],
        [0.16409038, 0.15541689, 0.26498961, ..., 0.22892763, 0.16970645,
         0.        ]]),
 'uuids': ['0e7fd0d4-9c69-4b08-ad14-ed912112171c',
  '59ffb624-5d6b-403c-af02-b82a08fccf5e',
  '0c4583b4-c3c4-47fe-a138-4ee30487b44d',
  'e30d24fd-6567-46b9-a6e2-23154ddcdbec',
  'c7089d45-b1ff-4204-9044-5f4dcd7f066f',
  '30dd5dfe-8bcb-47cd-9bbf-36d4f9a01c1d',
  'de5d5cab-27f4-4081-8007-98861b45efc2',
  '5cb53d2d-6ffd-4031-9ec5-f6997f0de076',
  'b04e48e3-9ad8-405e-8842-f9d6660e02f1',
  'dfeece4f-

In [6]:
names = [uuid_to_name[uuid] for uuid in results["uuids"]]
scores = results["scores"]

In [7]:
res = get_top_ks(names, 1-scores, ks=[1,3,5,10])

for k, acc in res:
    print(f"Top-{k}: {acc}")

Top-1: 0.5115346038114343
Top-3: 0.571715145436309
Top-5: 0.5747241725175527
Top-10: 0.59679037111334


In [14]:
for descriptor in ["Superpoint", "ALIKED", "DoGHardNet", "DISK"]:
    
    results = load_pickle(f"lightglue_scores_{descriptor.lower()}.pickle")
    names = [uuid_to_name[uuid] for uuid in results["uuids"]]
    scores = results["scores"]
    res = get_top_ks(names, 1-scores, ks=[1,3,5,10])
    print(f"Lightglue + {descriptor}")
    for k, acc in res:
        print(f"Top-{k}: {100 * acc:.2f}%")
    print()

Lightglue + Superpoint
Top-1: 51.15%
Top-3: 57.17%
Top-5: 57.47%
Top-10: 59.68%

Lightglue + ALIKED
Top-1: 39.22%
Top-3: 46.14%
Top-5: 48.35%
Top-10: 50.45%

Lightglue + DoGHardNet
Top-1: 50.55%
Top-3: 55.67%
Top-5: 57.17%
Top-10: 58.98%

Lightglue + DISK
Top-1: 35.81%
Top-3: 43.73%
Top-5: 46.24%
Top-10: 48.35%

